In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split


In [2]:
mon_laun_path = "C:/Users/zzhan/OneDrive/Рабочий стол/XGBTrial/Datasets/merged_dataset.csv"
"""
0+10
01+0
"""
mon_laun_df = pd.read_csv(mon_laun_path)


display(mon_laun_df)

,Unnamed: 0,transactionID,sourceAccountID,targetAccountID,timestamp,fan_in_bins_2-3,fan_in_bins_3-inf,fan_out_bins_2-3,fan_out_bins_3-inf,degree_in_bins_2-3,...,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,sourceAccount,targetAccount
0,0,1.0,8.0,9.0,10.0,0.0,6.0,0.0,3.0,0.0,...,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0,32088000F4580,18000F5340
1,1,12.0,13.0,14.0,22.0,0.0,10.0,0.0,5.0,0.0,...,245335,812ED62E0,4.01,US Dollar,4.01,US Dollar,Credit Card,0,18000F4FE0,245335812ED62E0
2,2,24.0,25.0,28.0,29.0,0.0,8.0,0.0,5.0,0.0,...,13327,810B0FB40,86.30,US Dollar,86.30,US Dollar,Credit Card,0,108005F0B50,13327810B0FB40
3,3,32.0,33.0,34.0,38.0,0.0,0.0,0.0,0.0,0.0,...,1688,8005FD430,159271.00,US Dollar,159271.00,US Dollar,ACH,0,10800058100,16888005FD430
4,4,39.0,40.0,42.0,43.0,0.0,0.0,0.0,0.0,0.0,...,1420,801008520,5069.19,US Dollar,5069.19,US Dollar,Cheque,0,1674800AC8E30,1420801008520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,681,8.0,29.0,19.0,15.0,1.0,12.0,0.0,5.0,0.0,...,2439,8017BF800,7.66,US Dollar,7.66,US Dollar,Credit Card,0,128000EC280,24398017BF800
682,682,13.0,13.0,13.0,19.0,0.0,8.0,0.0,3.0,0.0,...,36056,812ED6380,106.70,US Dollar,106.70,US Dollar,Credit Card,0,1080012FD90,36056812ED6380
683,683,8.0,2.0,22.0,6.0,1.0,12.0,0.0,5.0,0.0,...,2439,8017BF800,7.66,US Dollar,7.66,US Dollar,Credit Card,0,128000EC280,24398017BF800
684,684,28.0,8.0,17.0,14.0,0.0,3.0,0.0,3.0,2.0,...,1420,8006A3840,1210984.00,US Dollar,1210984.00,US Dollar,ACH,0,1800A5F020,14208006A3840


In [3]:
"""enriched_df = pd.read_csv("C:/Users/zzhan/OneDrive/Рабочий стол/XGBTrial/Datasets/enriched_pure.csv")"""
enriched_df = mon_laun_df

train_df_target = mon_laun_df[["Is Laundering"]].astype(int)
train_df_data = mon_laun_df.drop(columns=["Is Laundering"])

train_df_data = train_df_data.drop(columns=['Timestamp', 'Account', 'Account.1', 'Receiving Currency','Payment Currency','Payment Format','sourceAccount','targetAccount'])
X = train_df_data.to_numpy()
y = train_df_target.to_numpy()


In [4]:
print(len(X))
print(len(X[0]))
print(len(y))

686
53
686


In [5]:
train_df_data.dtypes


Unnamed: 0                     int64
transactionID                float64
sourceAccountID              float64
targetAccountID              float64
timestamp                    float64
fan_in_bins_2-3              float64
fan_in_bins_3-inf            float64
fan_out_bins_2-3             float64
fan_out_bins_3-inf           float64
degree_in_bins_2-3           float64
degree_in_bins_3-inf         float64
degree_out_bins_2-3          float64
degree_out_bins_3-inf        float64
scatter-gather_bins_2-3      float64
scatter-gather_bins_3-inf    float64
temp-cycle_bins_2-3          float64
temp-cycle_bins_3-inf        float64
source_fan_out               float64
source_deg_out               float64
source_ratio_out             float64
source_avg_col3_out          float64
source_sum_col3_out          float64
source_var_col3_out          float64
source_skew_col3_out         float64
source_kurtosis_col3_out     float64
source_fan_in                float64
source_deg_in                float64
s

In [6]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X, y)

y_pred = xgb_model.predict(X)

print(confusion_matrix(y, y_pred))


[[685   0]
 [  1   0]]


In [7]:
cancer = load_breast_cancer()

X = cancer.data
y = cancer.target

xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X, y)

y_pred = xgb_model.predict(X)

print(len(y))

569
